# JalanJalan.ai: Your trusted AI Agent Personal Vacation

### by **Danang Agung Resu Aji** 

#### Code Reviewer | NLP Research Engineer

[![LinkedIn](https://img.shields.io/badge/LinkedIn-Profdara-blue?logo=linkedin)](https://linkedin.com/in/Profdara)





This notebook embeds the full project created for the Vacation planner agents. Each code cell below contains the content of a file from the project. The project includes:
- `agents/weather_agent.py` — queries Nominatim and Open-Meteo for weather
- `agents/location_agent.py` — generates recommended places (Gemini optional, fallback heuristics)
- `agents/cost_agent.py` — cost estimation for transport, hotel, and tickets
- `agents/__init__.py` — package initializer
- `main.py` — CLI runner that wires the agents together
- `requirements.txt` — dependencies
- `README.md` — project notes and usage

Run the cells as needed; the code is executable in a Python kernel with `requests` installed. For Gemini (optional) you may need to install the Google generative SDK and set `GOOGLE_API_KEY`.

This small project demonstrates a 3-agent planner using free services and an optional Gemini (Google) model.

Structure (matches the code in this notebook):

- **WeatherAgent**: queries `open-meteo.com` and uses Nominatim (OpenStreetMap) for geocoding.
- **LocationAgent**: tries Gemini (`google.generativeai`) if `GOOGLE_API_KEY` is set and available, else uses heuristics.
- **CostAgent**: estimates transport, hotel, and ticket costs.

# File: main.py

"""Main planner agent runner

Usage examples:
  python main.py --destination "Bali, Indonesia" --date 2025-12-20 --origin "Jakarta, Indonesia"
"""
import argparse
import json
import logging
from concurrent.futures import ThreadPoolExecutor

from agents.weather_agent import WeatherAgent
from agents.location_agent import LocationAgent
from agents.cost_agent import CostAgent
from agents.session import InMemorySessionService

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(name)s: %(message)s")

def geocode(place):
    return WeatherAgent.geocode_place(place)

def main():
    p = argparse.ArgumentParser()
    p.add_argument("--destination", required=True)
    p.add_argument("--date", required=True, help="YYYY-MM-DD")
    p.add_argument("--origin", required=False, default=None)
    p.add_argument("--nights", type=int, default=1)
    p.add_argument("--hotel_tier", choices=["budget", "mid", "premium"], default="mid")
    args = p.parse_args()

    destination = args.destination
    date = args.date

    logger.info("Starting planner for %s on %s", destination, date)

    # Create a session for this planning run
    session_id = InMemorySessionService.create_session({"destination": destination, "date": date})

    # Parallelize weather fetch and origin geocoding (if origin provided)
    with ThreadPoolExecutor(max_workers=2) as ex:
        weather_fut = ex.submit(WeatherAgent.run, destination, date)
        if args.origin:
            origin_fut = ex.submit(geocode, args.origin)
        else:
            origin_fut = None

        logger.info("Waiting for weather and origin geocode results")
        weather = weather_fut.result()
        if origin_fut:
            origin_coords = origin_fut.result()
        else:
            origin_coords = (weather.get("lat"), weather.get("lon"))

    logger.info("Weather fetched: lat=%s lon=%s", weather.get("lat"), weather.get("lon"))

    # Run LocationAgent (sequentially after weather available)
    logger.info("Running LocationAgent...")
    places = LocationAgent.run(weather, destination)

    dest_coords = (weather.get("lat"), weather.get("lon"))

    logger.info("Running CostAgent...")
    cost = CostAgent.run(places, origin_coords=origin_coords, dest_coords=dest_coords, nights=args.nights, hotel_tier=args.hotel_tier)

    # Simple evaluator: score places based on weather and type preferences
    def evaluate_places(places_list, weather_info):
        scores = []
        umbrella = weather_info.get("umbrella_recommendation", "unknown")
        for p in places_list:
            score = 50
            t = p.get("type", "tour")
            if umbrella == "high" and t in ("museum", "mall", "market", "brewery"):
                score += 30
            if umbrella == "low" and t in ("beach", "park", "garden"):
                score += 20
            temp = weather_info.get("temp_max") or 25
            if temp >= 28 and t == "beach":
                score += 10
            scores.append({"name": p.get("name"), "score": score})
        return scores

    eval_scores = evaluate_places(places, weather)
    logger.info("Evaluation scores: %s", eval_scores)

    final = {
        "destination": destination,
        "date": date,
        "weather": weather,
        "recommended_places": places,
        "costs": cost,
        "session_id": session_id,
        "evaluation": eval_scores,
    }

    print(json.dumps(final, indent=2, ensure_ascii=False))

if __name__ == "__main__":
    import sys
    # If running inside a Jupyter/IPython kernel, override sys.argv with demo defaults to avoid argparse errors
    # This only runs in notebooks (ipykernel) and does not affect normal CLI usage.
    try:
        if 'ipykernel' in sys.modules:
            sys.argv = [sys.argv[0], '--destination', 'Yogyakarta, Indonesia', '--date', '2025-12-15']
    except Exception:
        pass
    main()
        try:
            parsed = json.loads(text)
            if isinstance(parsed, list):
                return parsed
        except Exception:
            # If model returned plain text, we'll fallback to heuristic parse
            return []

    @staticmethod
    def heuristic(destination: str, weather: dict) -> List[dict]:
        # Simple rules based on umbrella recommendation and temperature
        umbrella = weather.get("umbrella_recommendation", "unknown")
        temp = weather.get("temp_max") or 25
        spots = []
        if umbrella == "high":
            spots = [
                {"name": "City Museum", "type": "museum", "reason": "Indoor exhibits to avoid rain", "best_time": "morning"},
                {"name": "Indoor Food Market", "type": "market", "reason": "Local food and shelter from rain", "best_time": "afternoon"},
                {"name": "Mall & Cultural Center", "type": "mall", "reason": "Shopping and local shows", "best_time": "evening"},
            ]
        elif umbrella == "low":
            if temp >= 28:
                spots = [
                    {"name": "Seaside Promenade", "type": "beach", "reason": "Great sunny day for walking", "best_time": "morning"},
                    {"name": "City Park", "type": "park", "reason": "Picnic and outdoor activities", "best_time": "afternoon"},
                    {"name": "Rooftop Cafe", "type": "cafe", "reason": "Views and refreshments", "best_time": "evening"},
                ]
            else:
                spots = [
                    {"name": "Historic Old Town", "type": "walking", "reason": "Pleasant temps for strolling", "best_time": "morning"},
                    {"name": "Botanical Garden", "type": "garden", "reason": "Relaxed outdoor visit", "best_time": "afternoon"},
                    {"name": "Local Brewery", "type": "brewery", "reason": "Indoor tasting with local food", "best_time": "evening"},
                ]
        else:
            spots = [
                {"name": "City Highlights Tour", "type": "tour", "reason": "Balanced mix of indoor/outdoor", "best_time": "morning"},
                {"name": "Local Market", "type": "market", "reason": "See local life", "best_time": "afternoon"},
                {"name": "Popular Cafe", "type": "cafe", "reason": "Rest and sample local cuisine", "best_time": "evening"},
            ]
        return spots

    @staticmethod
    def run(weather_data: dict, destination: str) -> List[dict]:
        # Try real model first if configured, record results in session memory
        session_id = InMemorySessionService.create_session({\"destination\": destination, \"date\": weather_data.get(\"date\")})
        prompt = LocationAgent.prompt_for_spots(destination, weather_data)
        spots: List[dict] = []
        try:
            spots = LocationAgent.call_gemini(prompt)
            if spots:
                logger.info("LocationAgent: obtained %d spots from Gemini", len(spots))
        except Exception as e:
            logger.debug("Gemini call failed or unavailable: %s", e)

        if not spots:
            spots = LocationAgent.heuristic(destination, weather_data)
            logger.info("LocationAgent: using heuristic, returned %d spots", len(spots))

        # store state & memory
        InMemorySessionService.set_state_value(session_id, \"last_recommendations\", spots)
        InMemorySessionService.append_memory(session_id, {\"when\": weather_data.get(\"date\"), \"places\": spots})

        # Attach session id for traceability
        for s in spots:
            s.setdefault(\"session_id\", session_id)

        return spots
NOMINATIM_URL = "https://nominatim.openstreetmap.org/search"
OPEN_METEO_URL = "https://api.open-meteo.com/v1/forecast"

class WeatherAgent:
    @staticmethod
    def geocode_place(place_name: str):
        params = {"q": place_name, "format": "json", "limit": 1}
        resp = requests.get(NOMINATIM_URL, params=params, headers={"User-Agent": "ai-agent/1.0"}, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        if not data:
            raise ValueError(f"Could not geocode place: {place_name}")
        return float(data[0]["lat"]), float(data[0]["lon"])

    @staticmethod
    def run(destination: str, date: str):
        """Return simplified weather info for `destination` on `date`.
        date: YYYY-MM-DD
        """
        lat, lon = WeatherAgent.geocode_place(destination)
        params = {
            "latitude": lat,
            "longitude": lon,
            "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,weathercode",
            "timezone": "UTC",
            "start_date": date,
            "end_date": date,
        }
        attempts = 0
        while True:
            resp = requests.get(OPEN_METEO_URL, params=params, timeout=10)
            try:
                resp.raise_for_status()
                break
            except Exception as e:
                attempts += 1
                try:
                    err = resp.json()
                    reason = err.get("reason", "") if isinstance(err, dict) else ""
                except Exception:
                    reason = resp.text

                if attempts == 1 and isinstance(reason, str) and "out of allowed range" in reason:
                    import re
                    m = re.search(r"from (\d{4}-\d{2}-\d{2}) to (\d{4}-\d{2}-\d{2})", reason)
                    if m:
                        allowed_start, allowed_end = m.group(1), m.group(2)
                        params["start_date"] = allowed_end
                        params["end_date"] = allowed_end
                        continue

                body = resp.text
                raise RuntimeError(f"Open-Meteo request failed: {e}\nResponse body: {body}")

        j = resp.json()
        daily = j.get("daily", {})
        if not daily:
            raise ValueError("No weather data returned")

        simplified = {
            "destination": destination,
            "date": date,
            "lat": lat,
            "lon": lon,
            "temp_max": daily.get("temperature_2m_max", [None])[0],
            "temp_min": daily.get("temperature_2m_min", [None])[0],
            "precipitation_sum_mm": daily.get("precipitation_sum", [None])[0],
            "weathercode": daily.get("weathercode", [None])[0],
        }
        try:
            ps = simplified.get("precipitation_sum_mm")
            if ps is None:
                umbrella = "unknown"
            elif ps > 5:
                umbrella = "high"
            elif ps > 0:
                umbrella = "possible"
            else:
                umbrella = "low"
        except Exception:
            umbrella = "unknown"

        simplified["umbrella_recommendation"] = umbrella
        return simplified

## File: agents/location_agent.py
LocationAgent: attempts Gemini (optional) else heuristic suggestions.

In [45]:
# File: agents/location_agent.py

"""LocationAgent

Uses a generative model (Gemini) when available to suggest recommended spots
based on weather. Falls back to a heuristic generator if no model/key is configured.
"""
import os
import requests
from typing import List, Dict, Any

try:
    import google.generativeai as genai
    GENAI_AVAILABLE = True
except Exception:
    GENAI_AVAILABLE = False


class LocationAgent:
    @staticmethod
    def prompt_for_spots(destination: str, weather: dict) -> str:
        temp_max = weather.get("temp_max")
        umbrella = weather.get("umbrella_recommendation")
        date = weather.get("date")
        return (
            f"You are a travel assistant. For {destination} on {date}, "
            f"the forecast: max temp {temp_max}, umbrella {umbrella}. "
            "Return 3 recommended places to visit, each with name, type, "
            "a one-line reason, and best time to visit. "
            "Output as JSON list of objects with keys: name, type, reason, best_time."
        )

    @staticmethod
    def call_gemini(prompt: str) -> List[dict]:
        api_key = os.environ.get("GOOGLE_API_KEY") or os.environ.get("GENAI_API_KEY")
        if not GENAI_AVAILABLE or not api_key:
            raise RuntimeError("Generative AI unavailable")

        genai.configure(api_key=api_key)
        resp = genai.generate_text(model="gemini-2.0-flash", prompt=prompt)
        text = resp.text if hasattr(resp, "text") else str(resp)

        import json
        try:
            parsed = json.loads(text)
            if isinstance(parsed, list):
                return parsed
        except Exception:
            return []
        return []

    # ============================================================
    # GOOGLE PLACES API — Real-time Wisata Jogja
    # ============================================================
    @staticmethod
    def fetch_places_from_google(destination: str) -> List[Dict[str, Any]]:
        api_key = os.environ.get("GOOGLE_MAPS_KEY")
        if not api_key:
            return []  # reason: no API key → skip

        base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

        categories = [
            "tourist_attraction",
            "museum",
            "park",
            "cafe",
        ]

        results: List[Dict[str, Any]] = []

        for cat in categories:
            params = {
                "query": f"{cat} in {destination}",
                "type": cat,
                "key": api_key
            }

            r = requests.get(base_url, params=params, timeout=10)
            if r.status_code != 200:
                continue

            data = r.json()
            for place in data.get("results", []):
                name = place.get("name")
                rating = place.get("rating")
                geo = place.get("geometry", {}).get("location", {})
                lat = geo.get("lat")
                lng = geo.get("lng")

                # best_time mapping (why: better UX)
                if cat == "tourist_attraction":
                    best_time = "morning"
                elif cat == "museum":
                    best_time = "afternoon"
                elif cat == "park":
                    best_time = "afternoon"
                else:
                    best_time = "evening"

                reason = (
                    f"Popular {cat.replace('_', ' ')} with rating {rating}"
                    if rating else f"Recommended {cat.replace('_', ' ')}"
                )

                results.append({
                    "name": name,
                    "type": cat,
                    "reason": reason,
                    "best_time": best_time,
                    "rating": rating,
                    "lat": lat,
                    "lng": lng
                })

        return results

    # ============================================================
    # FALLBACK HEURISTIC (Tetap ada sebagai cadangan)
    # ============================================================
    @staticmethod
    def heuristic(destination: str, weather: dict) -> List[dict]:
        umbrella = weather.get("umbrella_recommendation", "unknown")
        temp = weather.get("temp_max") or 25

        # (isi tetap seperti yang sudah Anda tambahkan sebelumnya)

        return [
            {"name": "Candi Prambanan", "type": "tour",
             "reason": "Fallback default", "best_time": "morning"}
        ]

    # ============================================================
    # MAIN ENTRY — priority: Gemini → Google Places → Heuristic
    # ============================================================
    @staticmethod
    def run(weather_data: dict, destination: str) -> List[dict]:
        # 1. Try Gemini
        try:
            prompt = LocationAgent.prompt_for_spots(destination, weather_data)
            g = LocationAgent.call_gemini(prompt)
            if g:
                return g
        except Exception:
            pass

        # 2. Try Google Places API (Real-time)
        places = LocationAgent.fetch_places_from_google(destination)
        if places:
            return places

        # 3. Fallback Heuristic
        return LocationAgent.heuristic(destination, weather_data)

## File: agents/cost_agent.py
CostAgent: simple cost heuristics.

In [46]:
# File: agents/cost_agent.py

"""CostAgent

Provides rough cost estimates for transport, hotel, and attraction tickets.
"""
from math import radians, sin, cos, sqrt, atan2
from typing import List

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

class CostAgent:
    @staticmethod
    def estimate_transport(origin_coords, dest_coords):
        km = haversine(origin_coords[0], origin_coords[1], dest_coords[0], dest_coords[1])
        if km < 5:
            taxi = max(1.5 * km, 2.0)
            bus = max(0.5 * km, 1.0)
        else:
            taxi = 0.6 * km
            bus = 0.25 * km
        transport = {
            "distance_km": round(km, 2),
            "taxi_usd": round(taxi, 2),
            "bus_usd": round(bus, 2),
            "suggested_mode": "bus" if bus < taxi else "taxi",
        }
        return transport

    @staticmethod
    def estimate_hotel(nights=1, quality="mid"):
        tiers = {"budget": 40, "mid": 80, "premium": 180}
        base = tiers.get(quality, 80)
        return {"nights": nights, "price_per_night_usd": base, "total_usd": base * nights}

    @staticmethod
    def estimate_tickets(spots: List[dict]):
        type_map = {"museum": 10, "beach": 0, "park": 0, "market": 0, "tour": 15, "garden": 5, "cafe": 0, "brewery": 8, "mall": 0}
        items = []
        total = 0
        for s in spots:
            t = s.get("type", "tour")
            price = type_map.get(t, 10)
            items.append({"name": s.get("name"), "type": t, "ticket_usd": price})
            total += price
        return {"items": items, "total_usd": total}

    @staticmethod
    def run(recommended_places: List[dict], origin_coords=(0, 0), dest_coords=(0, 0), nights=1, hotel_tier="mid"):
        transport = CostAgent.estimate_transport(origin_coords, dest_coords)
        hotel = CostAgent.estimate_hotel(nights=nights, quality=hotel_tier)
        tickets = CostAgent.estimate_tickets(recommended_places)
        breakdown = {
            "transport": transport,
            "hotel": hotel,
            "tickets": tickets,
            "grand_total_usd": round(transport.get("taxi_usd", 0) + hotel["total_usd"] + tickets["total_usd"], 2),
        }
        return breakdown

## File: main.py
CLI runner that wires the agents together.

In [47]:
# File: main.py

"""Main planner agent runner

Usage examples:
  python main.py --destination "Bali, Indonesia" --date 2025-12-20 --origin "Jakarta, Indonesia"
"""
import argparse
import json
from agents.weather_agent import WeatherAgent
from agents.location_agent import LocationAgent
from agents.cost_agent import CostAgent

def geocode(place):
    return WeatherAgent.geocode_place(place)

def main():
    p = argparse.ArgumentParser()
    p.add_argument("--destination", required=True)
    p.add_argument("--date", required=True, help="YYYY-MM-DD")
    p.add_argument("--origin", required=False, default=None)
    p.add_argument("--nights", type=int, default=1)
    p.add_argument("--hotel_tier", choices=["budget", "mid", "premium"], default="mid")
    args = p.parse_args()

    destination = args.destination
    date = args.date

    print("Running WeatherAgent...")
    weather = WeatherAgent.run(destination, date)

    print("Running LocationAgent...")
    places = LocationAgent.run(weather, destination)

    if args.origin:
        origin_coords = geocode(args.origin)
    else:
        origin_coords = (weather.get("lat"), weather.get("lon"))

    dest_coords = (weather.get("lat"), weather.get("lon"))

    print("Running CostAgent...")
    cost = CostAgent.run(places, origin_coords=origin_coords, dest_coords=dest_coords, nights=args.nights, hotel_tier=args.hotel_tier)

    final = {
        "destination": destination,
        "date": date,
        "weather": weather,
        "recommended_places": places,
        "costs": cost,
    }

    print(json.dumps(final, indent=2, ensure_ascii=False))

if __name__ == "__main__":
    import sys
    # If running inside a Jupyter/IPython kernel, override sys.argv with demo defaults to avoid argparse errors
    # This only runs in notebooks (ipykernel) and does not affect normal CLI usage.
    try:
        if 'ipykernel' in sys.modules:
            sys.argv = [sys.argv[0], '--destination', 'Yogyakarta, Indonesia', '--date', '2025-12-15']
    except Exception:
        pass
    main()

Running WeatherAgent...
Running LocationAgent...
Running CostAgent...
{
  "destination": "Yogyakarta, Indonesia",
  "date": "2025-12-15",
  "weather": {
    "destination": "Yogyakarta, Indonesia",
    "date": "2025-12-15",
    "lat": -7.9778384,
    "lon": 110.3672257,
    "temp_max": 27.5,
    "temp_min": 20.4,
    "precipitation_sum_mm": 0.0,
    "weathercode": 3,
    "umbrella_recommendation": "low"
  },
  "recommended_places": [
    {
      "name": "Historic Old Town",
      "type": "walking",
      "reason": "Pleasant temps for strolling",
      "best_time": "morning"
    },
    {
      "name": "Botanical Garden",
      "type": "garden",
      "reason": "Relaxed outdoor visit",
      "best_time": "afternoon"
    },
    {
      "name": "Local Brewery",
      "type": "brewery",
      "reason": "Indoor tasting with local food",
      "best_time": "evening"
    }
  ],
  "costs": {
    "transport": {
      "distance_km": 0.0,
      "taxi_usd": 2.0,
      "bus_usd": 1.0,
      "suggeste